## Start

In [ ]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #ModifiedSources

### Links: 
___

In [6]:
query_text = """--sql
CREATE TABLE db1.all_installetion_points_parquet
(
    `report_date` Date,
    `address_uuid` String,
    `city` String,
    `country` String,
    `region` String,
    `created_at` String,
    `full_address` String,
    `parent_uuid` String,
    `installation_point_id` String
)
ENGINE = S3(
    'https://storage.yandexcloud.net/aggregated-data/all_installetion_points_parquet/year=*/month=*/day=*/*.parquet',
    'Parquet'
)
"""

ch.query_run(query_text)

""


In [ ]:
    CAST(`report_date` AS DATE) AS `report_date`,
    CAST(`address_uuid` AS UTF8) AS `address_uuid`,
    CAST(`city` AS UTF8) AS `city`,
    CAST(`country` AS UTF8) AS `country`,
    CAST(`region` AS UTF8) AS `region`,
    CAST(`created_at` AS UTF8) AS `created_at`,
    CAST(`full_address` AS UTF8) AS `full_address`,
    CAST(`parent_uuid` AS UTF8) AS `parent_uuid`,
    CAST(`installation_point_id` AS UTF8) AS `installation_point_id`,

In [8]:
query_text = """--sql
CREATE TABLE db1.all_installetion_points_parquet_ch
    (
    `report_date` Date,
    `address_uuid` String,
    `city` String,
    `country` String,
    `region` String,
    `created_at` String,
    `full_address` String,
    `parent_uuid` String,
    `installation_point_id` Int32
    )
    ENGINE = MergeTree()
    ORDER BY installation_point_id
"""

ch.query_run(query_text)

""


In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.all_installetion_points_parquet_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR TO db1.all_installetion_points_parquet_ch AS
    SELECT
        *
    FROM db1.all_installetion_points_parquet
    """

ch.query_run(query_text)

___
## Tools
___
### query


In [9]:
query_text = """--sql
    SELECT
        *
    FROM db1.all_installetion_points_parquet
    ORDER BY report_date desc
    limit 10
    """

ch.query_run(query_text)


,report_date,address_uuid,city,country,region,created_at,full_address,parent_uuid,installation_point_id
0,2025-09-14,71f30258-c9fd-466c-a263-ac779c2fd3c3,рабочий посёлок Вычегодский,Россия,Архангельская область,2023-02-22 13:52:35,"Россия, Архангельская область, муниципальное о...",0ec23d35-2389-4825-97c8-7684ee1ec9df,493771
1,2025-09-14,f972c26a-d6f8-40d7-bb89-2809de94a1ba,Тольятти,Россия,Самарская область,2025-03-20 12:22:35,"Россия, Самарская область, городской округ Тол...",fa611d41-57e3-409f-bfe8-10a020979412,926332
2,2025-09-14,cf9a30ce-9e14-42f6-82f4-b537b8f56825,Тюмень,Россия,Тюменская область,2025-07-21 07:04:41,"Россия, Тюменская область, городской округ Тюм...",808342e6-4f92-4a85-be96-82d3b1bd0418,1038752
3,2025-09-14,7c68a652-17a3-4ac7-8ef9-1d70130e4b34,Владивосток,Россия,Приморский край,2021-05-24 11:21:24,"Россия, Приморский край, Владивостокский город...",7dd94897-3e46-4363-96f9-0edc02cf0cff,129120
4,2025-09-14,abbfc21f-8005-44c9-a678-f2bdfacdf614,Казань,Россия,Республика Татарстан (Татарстан),2025-01-13 09:35:47,"Россия, Республика Татарстан (Татарстан), горо...",1954d293-8479-451c-b247-3fd3b0c1492c,887197
5,2025-09-14,958ade87-e58c-4f7a-a6c1-40dc76710026,Краснодар,Россия,Краснодарский край,2021-06-25 13:38:24,"Россия, Краснодарский край, муниципальное обра...",973c9d71-53c5-4af5-8a88-e3796ec3007a,143443
6,2025-09-14,81ed38af-4a60-4b17-8def-aa2ef8dfca4c,Иваново,Россия,Ивановская область,2020-01-22 06:26:36,"Россия, Ивановская область, городской округ Ив...",6be68774-19ef-4cd3-b749-c9f69ceb8c70,6622
7,2025-09-14,8dcf7d0f-5c2b-46b8-98c1-7d810b355f84,Калининград,Россия,Калининградская область,2022-05-25 08:17:06,"Россия, Калининградская область, городской окр...",47edad87-08ad-4c3f-b531-8269cba086b1,345990
8,2025-09-14,04de5446-c24d-4ed4-82bc-c1a9f69ffc32,Стерлитамак,Россия,Республика Башкортостан,2022-03-23 13:25:48,"Россия, Республика Башкортостан, городской окр...",b45457ab-14e2-42bc-ad36-e009b918d8a1,305458
9,2025-09-14,699800bd-6012-4737-9b2c-118193d1a3bf,Иваново,Россия,Ивановская область,2022-05-31 06:45:32,"Россия, Ивановская область, городской округ Ив...",99e21d9a-fb43-49fc-8587-25a921e36722,349285


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.all_installetion_points_parquet_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [5]:
query_text = """--sql
    DROP TABLE db1.all_installetion_points_parquet
    """

ch.query_run(query_text)

""


### drop ch

In [7]:
query_text = """--sql
    DROP TABLE db1.all_installetion_points_parquet_ch
    """

ch.query_run(query_text)

""


### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.all_installetion_points_parquet_mv
"""

ch.query_run(query_text)